## Hospital Scheduling Using Bin Packing Approach

### Objective:
The objective of this approach is to efficiently schedule surgeries in hospital operating rooms while considering surgery durations, room availability, and surgery priorities.

### Method:
1. **Model Parameters:**
    - Number of surgeries, types, rooms, and days.
    - Operating hours per day.
    - Surgery durations, priorities, and total surgeries of each type.

2. **Model Creation:**
    - Initialize Gurobi model.
    - Define decision variables `x` representing whether a surgery of a particular type is scheduled in a specific room on a given day.

3. **Objective Function:**
    - Maximize the total weighted priority of scheduled surgeries.

4. **Constraints:**
    - Limit the total duration of surgeries in each room on each day to the operating hours.
    - Ensure each surgery is assigned to at most one room on one day.
    - Limit the total number of surgeries of each type scheduled.
    - Add a variance based slack time at the end of each day (for each room) based on the sum of variances of surgeries scheduled.

5. **Solving the Model:**
    - Optimize the model to find the optimal surgery schedule using gurobi GRB

6. **Output**
    - Outputs a html visualisation via plotly to show the 5 day 3 room schedule, along with missed surgery frequencies.
    - Outputs 3 data frames showing OR utilizaition per day, schedule of surgeries, missed surgery frequencies.


In [75]:
from gurobipy import Model, GRB
import numpy as np

# Model paramaters
num_surgeries = 40
num_types = 3  # Assuming types are numbered 1 through 3
num_rooms = 3
num_days = 15
operating_hours = 8

# Surgery Distribution
surgery_durations = {1: 5, 2: 5, 3: 5}
surgery_variances = {1: 1.5, 2: 0.75, 3: 2.0}
surgery_priorities = {1: 25, 2: 20, 3: 30}
total_surgeries_of_each_type = {1: 20, 2: 10, 3: 10}

# Model
m = Model("surgery_scheduling")

# Variable
x = m.addVars(num_surgeries, num_types+1, num_rooms, num_days, vtype=GRB.BINARY, name="schedule")

factor = 1 # priority score adjustment factor (to be adjusted to alter impact of surgery priority)

# Objective: maximize surgeries scheduled * weighted priority
m.setObjective(sum(x[i,j,k,l] * factor * surgery_priorities[j] for i in range(num_surgeries)
                   for j in range(1, num_types+1) for k in range(num_rooms)
                   for l in range(num_days)), GRB.MAXIMIZE)

# Constraints: 
#surgery durations cant exceed 8 hours per day per operating room
for k in range(num_rooms):
    for l in range(num_days):
        m.addConstr(sum(x[i,j,k,l] * surgery_durations[j] for i in range(num_surgeries)
                        for j in range(1, num_types+1)) <= operating_hours, 
                    f"duration_room{k+1}_day{l+1}")

# Surgeries can only be assigned to a maximum of 1 room in 1 day
for i in range(num_surgeries):
    m.addConstr(sum(x[i,j,k,l] for j in range(1, num_types+1) for k in range(num_rooms)
                    for l in range(num_days)) <= 1, 
                f"assign_surgery{i+1}")

#Ensures that we are selecting from the portfolio of surgeries 
for j in range(1, num_types+1):
    m.addConstr(sum(x[i,j,k,l] for i in range(num_surgeries) for k in range(num_rooms)
                    for l in range(num_days)) <= total_surgeries_of_each_type[j],
                f"total_type{j}")

beta = 0.50

# Adjust the surgery duration constraints to incorporate pseudo-variance-based slack time
for k in range(num_rooms):
    for l in range(num_days):
        slack_time_adjustment = beta * sum(x[i, j, k, l] * np.sqrt(surgery_variances[j]) for i in range(num_surgeries)
                                    for j in range(1, num_types+1))
        m.addConstr(sum(x[i, j, k, l] * surgery_durations[j] for i in range(num_surgeries)
                        for j in range(1, num_types+1)) + slack_time_adjustment <= operating_hours,
                    f"variance_adjusted_duration_room{k+1}_day{l+1}")

# Solve
m.optimize()

#Visualisation Pre Processing
scheduled_surgeries = []
missed_surgeries = {j: total_surgeries_of_each_type[j] for j in range(1, num_types+1)}
slack_time_each_day = {(k, l): operating_hours for k in range(1, num_rooms+1) for l in range(1, num_days+1)}

for i in range(num_surgeries):
    for j in range(1, num_types+1):
        for k in range(num_rooms):
            for l in range(num_days):
                if x[i,j,k,l].X > 0.5:  # If surgery i of type j is scheduled in room k on day l
                    scheduled_surgeries.append((i+1, j, k+1, l+1))
                    missed_surgeries[j] -= 1
                    slack_time_each_day[(k+1, l+1)] -= surgery_durations[j]

# output scheduled surgeries
print("Scheduled Surgeries:")
for surgery in scheduled_surgeries:
    print(f"Surgery {surgery[0]} of type {surgery[1]} is scheduled in room {surgery[2]} on day {surgery[3]}.")

# output missed surgeries
print("\nMissed Surgeries:")
for j in missed_surgeries:
    print(f"Missed surgeries of type {j}: {missed_surgeries[j]}")

# Output slack time for each room on each day
print("\nSlack Time (Hours) Each Day in Each Room:")
for k in range(1, num_rooms+1):
    for l in range(1, num_days+1):
        print(f"Room {k}, Day {l}: {slack_time_each_day[(k, l)]} hours")


Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 58 rows, 1500 columns and 4500 nonzeros
Model fingerprint: 0xeebc85d3
Variable types: 0 continuous, 1500 integer (1500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [2e+01, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 350.0000000
Presolve removed 15 rows and 375 columns
Presolve time: 0.03s
Presolved: 43 rows, 1125 columns, 3375 nonzeros
Variable types: 0 continuous, 1125 integer (1125 binary)
Found heuristic solution: objective 410.0000000

Root relaxation: cutoff, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | I

In [76]:
# Define a custom sorting key function
def custom_sort(surgery):
    # Extract relevant information
    surgery_id, type_id, room, day = surgery
    
    # Get duration and priority of the surgery
    duration = surgery_durations[type_id]
    priority = surgery_priorities[type_id]
    
    # Return a tuple for sorting
    return (day, duration, priority)

# Sort the scheduled surgeries
scheduled_surgeries_sorted = sorted(scheduled_surgeries, key=custom_sort)

# Scheduled Surgeries Output Transformation with sorting
scheduled_surgeries_data = []
for surgery in scheduled_surgeries_sorted:
    scheduled_surgeries_data.append({
        "Day": surgery[3],
        "Room": surgery[2],
        "Type": surgery[1],
        'Priority': surgery_priorities[ surgery[1]],
        "Surgery ID": surgery[0],       
    })

# Missed Surgeries Output Transformation
missed_surgeries_data = [{"Type": j, "Missed Count": missed_surgeries[j]} for j in missed_surgeries]


In [77]:
import pandas as pd

#df operations/groupbys
df_scheduled = pd.DataFrame(scheduled_surgeries_data)
df_scheduled_summary = df_scheduled.groupby(['Day', 'Room', 'Type']).size().reset_index(name='Count')

df_missed = pd.DataFrame(missed_surgeries_data)
df_missed["Type"] =df_missed["Type"].astype(str) #for graphing purposes

# Adding a 'Duration' column to df_scheduled based on the surgery type
df_scheduled['Duration'] = df_scheduled['Type'].apply(lambda x: surgery_durations[x])

# Calculate the total duration (utilization) of each OR by day
or_utilization = df_scheduled.groupby(['Room', 'Day'])['Duration'].sum().reset_index(name='Total Hours Used')

# Assuming 8 hours of available operating time per day
or_utilization['Total Hours Available'] = 8
or_utilization['Utilization (%)'] = (or_utilization['Total Hours Used'] / or_utilization['Total Hours Available']) * 100


In [78]:
df_missed

,Type,Missed Count
0,1,2
1,2,8
2,3,0


In [79]:
df_scheduled

,Day,Room,Type,Priority,Surgery ID,Duration
0,1,3,1,25,11,5
1,1,1,3,30,1,5
2,1,2,3,30,6,5
3,2,3,1,25,12,5
4,2,1,3,30,2,5
5,2,2,3,30,7,5
6,3,2,1,25,8,5
7,3,3,1,25,13,5
8,3,1,3,30,3,5
9,4,2,1,25,9,5


In [80]:
or_utilization

,Room,Day,Total Hours Used,Total Hours Available,Utilization (%)
0,1,1,5,8,62.5
1,1,2,5,8,62.5
2,1,3,5,8,62.5
3,1,4,5,8,62.5
4,1,5,5,8,62.5
5,2,1,5,8,62.5
6,2,2,5,8,62.5
7,2,3,5,8,62.5
8,2,4,5,8,62.5
9,2,5,5,8,62.5


In [81]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo

# Create subplots for each operating room and a summary subplot
fig = go.Figure()
# Assuming num_types is the total number of surgery types
num_types = 3

# Add subplots for each operating room
for room in sorted(df_scheduled['Room'].unique()):
    room_data = df_scheduled[df_scheduled['Room'] == room].groupby('Day').agg({'Duration': 'sum', 'Type': 'size'})
    
    # Calculate text for the bar labels
    text = [f"Total: {count}" for count in room_data['Type']]
    
    hover_text = []
    for day in room_data.index:
        day_data = df_scheduled[(df_scheduled['Room'] == room) & (df_scheduled['Day'] == day)]
        type_counts = [day_data[day_data['Type'] == i]['Type'].count() for i in range(1, num_types + 1)]
        hover_text.append(", ".join([f"Type {i+1}: {count}" for i, count in enumerate(type_counts)]))
    
    fig.add_trace(go.Bar(x=room_data.index, y=room_data['Duration'], name=f'Room {room}',
                         hoverinfo='text', text=text, hovertext=hover_text, textposition='auto'))


# Add summary subplot
summary_data = df_scheduled.groupby('Day').agg({'Duration': 'sum', 'Type': 'size'})
text_summary = [f"Total: {count}" for count in summary_data['Type']]
hover_text_summary = []
for day in summary_data.index:
    day_data = df_scheduled[df_scheduled['Day'] == day]
    type_counts = [day_data[day_data['Type'] == i]['Type'].count() for i in range(1, num_types + 1)]
    hover_text_summary.append(", ".join([f"Type {i+1}: {count}" for i, count in enumerate(type_counts)]))

fig.add_trace(go.Bar(x=summary_data.index, y=summary_data['Duration'], name='Total Hours Used/# of surgeries',
                     marker_color='black', opacity=0.3,
                     hoverinfo='text', text=text_summary, hovertext=hover_text_summary, textposition='auto'))

# Update layout
fig.update_layout(title_text='Operating Room Utilization by Day',
                  xaxis_title="Day",
                  yaxis_title="Hours",
                  legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
                  barmode='group',
                  bargap=0.2)

# Create bar chart for missed surgeries by type
fig_missed = px.bar(df_missed, x='Type', y='Missed Count', text='Missed Count',
                    title='Missed Surgeries by Type',
                    labels={'Missed Count': 'Number of Missed Surgeries', 'Type': 'Surgery Type'})

# Add line for 8-hour operating limit
fig.add_hline(y=8, line_dash="dot", 
              annotation_text="8-hour limit", 
              annotation_position="bottom right")

# Update x-axis category order
fig_missed.update_xaxes(categoryorder='array', categoryarray=[1, 2, 3])

# Update layout
fig_missed.update_layout(xaxis_title="Surgery Type", yaxis_title="Number of Missed Surgeries")

# Save HTML content of both plots separately
html_content_fig = fig.to_html(full_html=False)
html_content_fig_missed = fig_missed.to_html(full_html=False)

# Combine the HTML content into one file
combined_html_content = html_content_fig + html_content_fig_missed

# Save the combined HTML content to a file
with open('operating_room_utilization_combined_2.0.html', 'w') as f:
    f.write(combined_html_content)
